<a href="https://colab.research.google.com/github/Shyam6002/Gen_AI_2025/blob/main/chat_bot_and_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# Install dependencies
!pip install transformers torch

# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load pretrained DialoGPT model
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def chat():
    print("Chatbot: Hello! Type 'exit' to end the conversation.")
    chat_history = torch.tensor([])

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Tokenize user input
        input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt")

        # Append to chat history
        chat_history = torch.cat([chat_history, input_ids], dim=-1) if chat_history.numel() > 0 else input_ids

        # Generate a response
        output_ids = model.generate(chat_history, max_length=100, pad_token_id=tokenizer.eos_token_id)
        response = tokenizer.decode(output_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

        print(f"Chatbot: {response}")

# Start conversation
chat()

Chatbot: Hello! Type 'exit' to end the conversation.
You: shyam
Chatbot: I'm not sure if you're serious, but I'm pretty sure that's a reference to the movie The Shining.
You: tharique
Chatbot: thariqueshyam
You: vishwa
Chatbot: thariquevishwavishwa
You: do u know vishwa bhai
Chatbot: ishwado u know vishwa bhaiI know vishwa bhai
You: DBS
Chatbot: thariquevishwado u know vishwa bhaiDBSI know vishwa bhai
You: exit
Chatbot: Goodbye!


In [ ]:
# Install dependencies
!pip install torch torchvision gradio

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import gradio as gr

# Define Generator
class Generator(nn.Module):
    def __init__(self, input_dim=100, output_dim=784):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

# Define Discriminator
class Discriminator(nn.Module):
    def __init__(self, input_dim=784):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Load MNIST Dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

# Initialize models & optimizers
generator = Generator()
discriminator = Discriminator()
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)

# Train GAN
epochs = 10
for epoch in range(epochs):
    for real_images, _ in data_loader:
        real_images = real_images.view(real_images.size(0), -1)
        batch_size = real_images.size(0)

        real_labels = torch.ones(batch_size, 1)
        fake_labels = torch.zeros(batch_size, 1)

        # Train Discriminator
        optimizer_D.zero_grad()
        output_real = discriminator(real_images)
        loss_real = criterion(output_real, real_labels)

        noise = torch.randn(batch_size, 100)
        fake_images = generator(noise)
        output_fake = discriminator(fake_images)
        loss_fake = criterion(output_fake, fake_labels)

        loss_D = loss_real + loss_fake
        loss_D.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        noise = torch.randn(batch_size, 100)
        fake_images = generator(noise)
        output = discriminator(fake_images)
        loss_G = criterion(output, real_labels)
        loss_G.backward()
        optimizer_G.step()

    print(f"Epoch {epoch+1}: Loss D={loss_D.item()}, Loss G={loss_G.item()}")

# Gradio Interface for Image Generation
def generate_image(input_noise):
    input_tensor = torch.tensor([float(i) for i in input_noise.split(",")]).float()
    generated_image = generator(input_tensor).detach().numpy().reshape(28, 28)
    return generated_image

interface = gr.Interface(
    fn=generate_image,
    inputs=gr.Textbox(label="Enter 100 comma-separated values"),
    outputs=gr.Image(type="numpy"),
)

# Launch Gradio App
interface.launch()